# Building a CABLE NEWS NETWORK

In [1]:
import os
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import numpy as np
import annabel as anna
import cnn_functions as cnn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
layers = tf.keras.layers

In [2]:
#uploading ATTPC data image file and separating it into events (128x128 images) and targets (labels 0,1, and 2)
imgs = h5py.File('allimgs.h5', 'r')
events = np.asarray(imgs['imgs'])
targets = np.asarray(imgs['targets'])
#print(evts.shape, track.shape)

In [3]:
#normalizing the images so that the values are between -1 and 1
events = (events - 127.5) / 127.5
#adding a 3-value third dimension to each image so the model can accept it
events = np.expand_dims(events, axis=3)
events = np.repeat(events, 3, axis=3)

# Some checks to make sure the data has been correctly preprocessed
assert events.shape[1:] == (128, 128, 3), 'Real data has incorrect shape'
assert events.min() >= -1 and events.max() <= 1, 'Real data is not in the range [-1, 1]'

In [4]:
#shuffling the data and splitting it into training and testing sets
train_events, test_events, train_targets, test_targets = train_test_split(events, targets, test_size=0.25, random_state=24)
#print(train_events.shape, test_events.shape, train_targets.shape, test_targets.shape)

input_shape = train_events.shape[1:]  #should be (128, 128, 3)
num_classes = len(np.unique(train_targets))   #should be 3

In [5]:
#builds and compiles a model with the given specifications
model = cnn.build_pretrained_vgg_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
#checkpointing weights for each epoch
cp_dir = './model-checkpoints/'

if not os.path.exists(cp_dir):   #makes the directory if it doesn't exist
    os.makedirs(cp_dir)
    
cp_path = os.path.join(cp_dir, 'weights.epoch.{epoch:02d}')
cp_callback = tf.keras.callbacks.ModelCheckpoint(cp_path, save_weights_only=True)

In [8]:
#training the model and plotting its learning curve
history = model.fit(train_events, train_targets, batch_size=64, epochs=20, validation_split=0.2, callbacks=[cp_callback])
cnn.plot_learning_curve(history)

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
#evaluating the model at the optimal epoch
best_epoch = 5
best_cp_path = os.path.join(cp_dir, 'weights.epoch.{:02d}'.format(best_epoch))
model.load_weights(best_cp_path)

test_loss, test_accuracy = model.evaluate(test_events, test_targets, verbose=0)
print('Test accuracy: {:.04}'.format(test_accuracy))